In [41]:
import pandas as pd
from bs4 import BeautifulSoup
import glob
from selenium import webdriver
import requests 
import os
import time
import numpy as np

receipt_data_file = glob.glob('C:/Users/Playdata/Desktop/shinhancard/shinhancard_*.html') # 지정 폴더안에 있는 모든 파일을 불러내기 위함

total_df = []          # 전체 DataFrame 모음
total_df_region = []    # 지역별 DataFrame 모음
sheet_nm = []          # Excel Sheet Name 모음

# 위의 'total_df'와 'sheet_nm'에 데이터를 삽입하기 위한 for 구문
for url in receipt_data_file :
    driver = webdriver.Chrome('c:/chromedriver.exe')
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    title_tag = soup.select('head > title')
    title = title_tag[0].text

    # 암호된 문서일 경우 if 구문으로 들어감
    if title == '' :
        pw = '950730'
        driver.find_element_by_id('password').send_keys(pw)
        driver.find_element_by_xpath('//*[@id="aform"]/span/input').click()

    time.sleep(5)
    
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')

    # 암호된 문서가 아닐 경우 아래있는 코드로 바로 들어감
    tables = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr')
    tables_middleSum = soup.select('div#email02 > table.tblList.cardUseList > tbody > tr.middleSum')

    for n in range(len(tables_middleSum )) :
        tables.remove(tables_middleSum[n])
    
    # 명세서에서 추출한 데이터 string으로 변환하여 분리
    seperate_tables = []                          
    for n in range(len(tables)) :
        table = str(tables[n].text).split('\n')
        seperate_tables = seperate_tables + table

    # seperate_tables에서 '사용날짜' 추출   
    date_list = []
    for n in range(1,len(seperate_tables), 12) :
        date = seperate_tables[n]
        date_list.append(date)

    # seperate_tables에서 '사용처' 데이터만 추출    
    store_list = []
    for n in range(3,len(seperate_tables),12) :
        store = seperate_tables[n].replace(" ", "")
        store_list.append(store)

    # seperate_tables에서 추출된 '사용처' 데이터 --> 카카오 지도 API --> '사용처'에 대한 주소 추출      
    location_list = []
    def create_url(search_text):
        headers = {'Authorization': 'KakaoAK 3e8ea332f0cdef6293383ac7bf81a6fc'}
        base = "https://dapi.kakao.com/v2/local/search/keyword.json"
        param_query = "?query=" +search_text
    #     print(f'base + param_query: {base + param_query}')
        respond = requests.get(base + param_query,headers=headers)
        return respond.json()

    for n in range(len(store_list)) :
        store_name = store_list[n]
        result = create_url(store_name)
        tmp1 = result['documents']
        try :
            tmp2 = tmp1[0]
            tmp3 = tmp2['address_name']
            location_list.append(tmp3)
        except :
            tmp2 = np.nan
            location_list.append(tmp2)

    # seperate_tables에서 '사용금액' 데이터만 추출  
    price_list = [] 
    for n in range(4,len(seperate_tables),12) :
        price = seperate_tables[n]
        price_replace = int(price.replace(',',''))
        price_list.append(price_replace)
    
    # 추출한 데이터로 DataFrame 변환
    table_data = {'사용날짜':date_list, '사용처':store_list, '사용처주소':location_list,  '사용금액':price_list }
    
    df = pd.DataFrame(table_data, columns=['사용날짜', '사용처', '사용처주소', '사용금액'])
    df_sum = pd.DataFrame(df.sum(), columns=['합계'])
    df_total = df.append(df_sum.T)
    df_total.loc['합계', '사용날짜'] = '전체'
    df_total.loc['합계', '사용처'] = '전체'
    df_total.loc['합계', '사용처주소'] = '전체'
    total_df.append(df_total)   

############################################################################################################

    df_disap_nan = df.dropna()
        
    df_disap_nan_locli = list(df_disap_nan['사용처주소'])
    location_list_1 = [' '.join(address.split()[:2]) for address in df_disap_nan_locli]
    
    price_list_1 = list(df_disap_nan['사용금액'])
    
    table_data_1 = {'사용지역':location_list_1,  '사용금액':price_list_1}  
    df1 = pd.DataFrame(table_data_1, columns=['사용지역', '사용금액'])
    
############################################################################################################# 









#############################################################################################################

    tables_date = soup.select('body > div.wrap.check > div.content > div.receiveDate > span.date')
    nm = str(tables_date[0].text)
    sheet_nm.append(nm)
    
    driver.close()

# 모아진 DataFrame을 Excel로변환
folder = 'C:/Users/Playdata/Desktop/Card'

if not os.path.exists(folder) :
    os.makedirs(folder)
    
excel_writer = pd.ExcelWriter('C:/Users/Playdata/Desktop/Card/CardUseList.xlsx', engine='xlsxwriter')
ToExcel_data =[]

for n in range(len(total_df)) :
        ToAppend_df = total_df[n].to_excel(excel_writer, index = True, sheet_name = sheet_nm[n])
        ToExcel_data.append(ToAppend_df)
        
excel_writer.save()

In [238]:
df_disap_nan = df.dropna()
df_disap_nan_locli = list(df_disap_nan['사용처주소'])
location_list_1 = [ ' '.join(address.split()[:2]) for address in df_disap_nan_locli]
price_list_1 = list(df_disap_nan['사용금액'])

table_data_1 = {'사용지역':location_list_1,  '사용금액':price_list_1}

df1 = pd.Series({'사용금액':price_list_1}, index = location_list_1)
df1

서울 용산구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 노원구     NaN
서울 서초구     NaN
서울 노원구     NaN
경북 포항시     NaN
서울 노원구     NaN
서울 동대문구    NaN
서울 노원구     NaN
서울 성북구     NaN
dtype: object

In [235]:
df_disap_nan = df.dropna()
df_disap_nan_locli = list(df_disap_nan['사용처주소'])
location_list_1 = [ ' '.join(address.split()[:2]) for address in df_disap_nan_locli]
price_list_1 = list(df_disap_nan['사용금액'])

table_data_1 = {'사용지역':location_list_1,  '사용금액':price_list_1}  

df1 = pd.DataFrame(table_data_1, columns=['사용지역','사용금액'])

df1_region_only = df1.drop_duplicates(subset = '사용지역')

name_index = df1_region_only['사용지역']
data_in_name = df1.rename(name_index)
data_in_name.drop(['사용지역'], axis = 1)

name_index = df1_region_only['사용지역']
data_in_only_name = df1_region_only.rename(name_index)
data_in_only_name.drop(['사용지역'], axis = 1)

for name in list(name_index) :
    tmp_data = data_in_name.loc[name]['사용금액']
    tmp_data_sum = tmp_data.sum()
    data_in_only_name.loc[name]['사용금액'] = tmp_data_sum
    
    print(type(tmp_data))
    if type(tmp_data) == 'pandas.core.series.Series' :  # 시리즈이면
        price_2_list = tmp_data.values
        price_2 = list(map(int, price_2_list))
        price_sum = sum(price_2)
        data_in_only_name.loc[name]['사용금액'] = price_sum
    else :
        data_in_only_name['사용금액'][name] = tmp_data.sum()

data_in_only_name

<ipython-input-235-f65707386de4>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_in_only_name.loc[name]['사용금액'] = tmp_data_sum


,사용지역,사용금액
서울 용산구,서울 용산구,1000
서울 노원구,서울 노원구,5000
서울 서초구,서울 서초구,66000
경북 포항시,경북 포항시,6000
서울 동대문구,서울 동대문구,3000
서울 성북구,서울 성북구,5800


In [147]:
a = list(name_index)
a

['서울 용산구', '서울 노원구', '서울 서초구', '경북 포항시', '서울 동대문구', '서울 성북구']

In [218]:
for name in a :
    tmp_data = data_in_name.loc[name]['사용금액']
    b = tmp_data.sum()
    print(f'name: {name} 총합: {b}')

name: 서울 용산구 총합: 1000
name: 서울 노원구 총합: 61810
name: 서울 서초구 총합: 66000
name: 경북 포항시 총합: 6000
name: 서울 동대문구 총합: 3000
name: 서울 성북구 총합: 5800


In [ ]:
c = tmp_data.values
c

In [176]:
type(tmp_data)

pandas.core.series.Series

In [170]:
price_2 = list(map(int, c))
price_2

price_sum = sum(price_2)
price_sum

61810

In [232]:
df1

,사용지역,사용금액
0,서울 용산구,1000
1,서울 노원구,5000
2,서울 노원구,2300
3,서울 노원구,2400
4,서울 노원구,2340
5,서울 노원구,8500
6,서울 노원구,8500
7,서울 노원구,1400
8,서울 노원구,2400
9,서울 노원구,2450


In [233]:
df1_region_only

,사용지역,사용금액
0,서울 용산구,1000
1,서울 노원구,5000
13,서울 서초구,66000
15,경북 포항시,6000
17,서울 동대문구,3000
19,서울 성북구,5800


In [236]:
name_index

0      서울 용산구
1      서울 노원구
13     서울 서초구
15     경북 포항시
17    서울 동대문구
19     서울 성북구
Name: 사용지역, dtype: object